Inspection of the entire CMC Human clinical data set and its subset that is relevant to our BSM study.

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm
import synapseclient
import fwsel

In [2]:
syn = synapseclient.login()
wdir = '/big/results/bsm/2020-08-05-cmc-clinical'
cmc_clinical_syn = syn.get('syn2279441', downloadLocation=wdir, ifcollision='overwrite.local')
cmc_clinical = pd.read_csv(cmc_clinical_syn.path, index_col='Individual ID')

Welcome, Attila Jones!



In [3]:
samples = pd.read_csv('/big/results/bsm/2020-05-18-processed-samples/all_samples.csv')
individuals = list(set(samples['Individual ID']))
individuals.sort()
print(len(samples), 'samples from', len(individuals), 'individuals')

116 samples from 95 individuals


In [4]:
clin = cmc_clinical.loc[individuals, :]
clin.to_csv(wdir + '/all_samples_clinical.csv')
clin

,Individual Notes,Institution,Brain ID,SCZ Pair,BP Pair,Changed (used Affy phenotype),Reported Gender,Sex,Ethnicity,Race,...,Toxicology Screen: Moodstab,Toxicology Screen: Benzos,Toxicology Screen: Nicotine/Cotinine,Toxicology Screen: THC,Toxicology Screen: Cocaine,Toxicology Screen: Opiates,otherDiagnosis,otherMedicalDetail,mannerOfDeath,PMICertain
Individual ID,,,,,,,,,,,,,,,,,,,,,
CMC_MSSM_027,NaN,MSSM,NaN,NaN,NaN,NaN,Male,XY,Caucasian,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CMC_MSSM_033,NaN,MSSM,NaN,NaN,NaN,NaN,Female,XX,African-American,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CMC_MSSM_055,NaN,MSSM,NaN,NaN,NaN,NaN,Male,XY,Caucasian,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CMC_MSSM_056,NaN,MSSM,NaN,NaN,NaN,NaN,Female,XX,Caucasian,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CMC_MSSM_063,NaN,MSSM,NaN,NaN,NaN,NaN,Male,XY,Caucasian,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CMC_PITT_098,NaN,Pitt,NaN,Pitt 16,NaN,NaN,Male,XY,Caucasian,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CMC_PITT_101,NaN,Pitt,NaN,Pitt 8,NaN,NaN,Male,XY,Caucasian,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CMC_PITT_113,NaN,Pitt,NaN,NaN,NaN,NaN,Male,XY,Caucasian,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
clin = clin.dropna(axis=1)
clin.columns

Index(['Institution', 'Reported Gender', 'Sex', 'Ethnicity', 'ageOfDeath',
       'PMI (in hours)', 'Dx', 'Year of Autopsy', 'Benzodiazepines',
       'Anticonvulsants', 'AntipsychTyp', 'AntipsychAtyp', 'Antidepress',
       'Lithium', 'Tobacco', 'Alcohol', 'Illicitsub', 'causeOfDeath',
       'Hyperten', 'DiabetesInsDep', 'DiabetesNonInsDep'],
      dtype='object')

In [6]:
clin = clin.drop(columns='Reported Gender')
clin = clin.rename(columns={'PMI (in hours)': 'PMI', 'Year of Autopsy': 'YearOfAutopsy'})
clin = clin.astype('category')
clin = clin.astype({'ageOfDeath': np.float32, 'PMI': np.float32, 'YearOfAutopsy': np.float32})
clin['Dx'] = [np.int(y == 'SCZ') for y in clin['Dx']]
clin

,Institution,Sex,Ethnicity,ageOfDeath,PMI,Dx,YearOfAutopsy,Benzodiazepines,Anticonvulsants,AntipsychTyp,AntipsychAtyp,Antidepress,Lithium,Tobacco,Alcohol,Illicitsub,causeOfDeath,Hyperten,DiabetesInsDep,DiabetesNonInsDep
Individual ID,,,,,,,,,,,,,,,,,,,,
CMC_MSSM_027,MSSM,XY,Caucasian,66.0,12.100000,1,1994.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,-1.0,-1.0,-1.0
CMC_MSSM_033,MSSM,XX,African-American,81.0,4.000000,0,1996.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,1.0,1.0,0.0,1.0
CMC_MSSM_055,MSSM,XY,Caucasian,66.0,23.900000,0,2005.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
CMC_MSSM_056,MSSM,XX,Caucasian,67.0,16.500000,0,2005.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,1.0,1.0,1.0,0.0
CMC_MSSM_063,MSSM,XY,Caucasian,72.0,12.000000,0,1996.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,5.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CMC_PITT_098,Pitt,XY,Caucasian,56.0,14.500000,0,1997.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,1.0,1.0,0.0,0.0
CMC_PITT_101,Pitt,XY,Caucasian,42.0,26.100000,0,1997.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0
CMC_PITT_113,Pitt,XY,Caucasian,48.0,16.600000,0,2000.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [7]:
clin.columns

Index(['Institution', 'Sex', 'Ethnicity', 'ageOfDeath', 'PMI', 'Dx',
       'YearOfAutopsy', 'Benzodiazepines', 'Anticonvulsants', 'AntipsychTyp',
       'AntipsychAtyp', 'Antidepress', 'Lithium', 'Tobacco', 'Alcohol',
       'Illicitsub', 'causeOfDeath', 'Hyperten', 'DiabetesInsDep',
       'DiabetesNonInsDep'],
      dtype='object')

In [8]:
best = fwsel.forward_selected(clin, 'Dx')

In [10]:
best.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     Dx   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.560
Method:                 Least Squares   F-statistic:                     6.976
Date:                Wed, 05 Aug 2020   Prob (F-statistic):           2.56e-10
Time:                        22:51:34   Log-Likelihood:                -9.8487
No. Observations:                  95   AIC:                             61.70
Df Residuals:                      74   BIC:                             115.3
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   56.4145     13.090      4.310      0.000      30.332      82.497
AntipsychAtyp[T.1.0]         0.3949      0.079      5.010      0.000       0.238       0.552
Alcohol[T.0.0]               0.7332      0.366      2.001      0.049       0.003       1.463
Alcohol[T.1.0]               0.9011      0.341      2.646      0.010       0.223       1.580
AntipsychTyp[T.1.0]          0.5118      0.093      5.527      0.000       0.327       0.696
Ethnicity[T.Caucasian]      -0.0273      0.094     -0.292      0.771      -0.214       0.159
Ethnicity[T.Hispanic]       -0.9276      0.241     -3.847      0.000      -1.408      -0.447
DiabetesInsDep[T.0.0]        0.1343      0.092      1.456      0.150      -0.049       0.318
DiabetesInsDep[T.1.0]       -0.2653      0.132     -2.012      0.048      -0.528      -0.003
Institution[T.Pitt]         -0.2753      0.092     -2.996      0.004      -0.458      -0.092
Benzodiazepines[T.1.0]      -0.1897      0.098     -1.931      0.057      -0.385       0.006
causeOfDeath[T.1.0]         -0.3556      0.327     -1.087      0.281      -1.008       0.297
causeOfDeath[T.2.0]         -0.0079      0.402     -0.020      0.984      -0.809       0.793
causeOfDeath[T.3.0]          0.0903      0.407      0.222      0.825      -0.720       0.901
causeOfDeath[T.4.0]         -0.3144      0.393     -0.800      0.426      -1.097       0.468
causeOfDeath[T.5.0]         -0.1651      0.331     -0.499      0.619      -0.825       0.494
DiabetesNonInsDep[T.0.0]    -0.1263      0.089     -1.419      0.160      -0.304       0.051
DiabetesNonInsDep[T.1.0]    -0.0047      0.101     -0.047      0.963      -0.206       0.197
Sex[T.XXY]                  -0.0542      0.334     -0.163      0.871      -0.719       0.610
Sex[T.XY]                   -0.1184      0.076     -1.555      0.124      -0.270       0.033
YearOfAutopsy               -0.0281      0.007     -4.302      0.000      -0.041      -0.015
PMI                          0.0058      0.003      2.307      0.024       0.001       0.011
==============================================================================
Omnibus:                        8.057   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.018   Jarque-Bera (JB):                8.359
Skew:                          -0.523   Prob(JB):                       0.0153
Kurtosis:                       4.008   Cond. No.                     3.11e+19
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.93e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
%connect_info